In [42]:

import h5py
import numpy as np
import scipy.ndimage as nd
import pickle
import glob
import os
import shutil

filters=[{'name': 'uniform_filter', 'size': 4, 'axes': (1,2), 'mode' : 'wrap'},
                {'name': 'zoom', 'zoom': (1, 0.25, 0.25), 'mode' : 'grid-wrap'}]
path = '/volume1/scratch/share_dir/peppe/'
read_folder = 'data'
write_folder = 'data_filter'
filename = 'MSH-Fields_000001.h5'
# Get all filenames in the read_folder
all_filenames = glob.glob(f'{path}{read_folder}/*.h5')
filenames_list = [os.path.basename(f) for f in all_filenames]

for filename in filenames_list:
    print(f"Processing {read_filename}")
    print(f"Writing to {write_filename}")
    read_filename = f'{path}{read_folder}/{filename}'
    write_filename = f'{path}{write_folder}/{filename}.pkl'
    # Load the file
    verbose=False
    data = {}
    with h5py.File(read_filename, 'r') as n:
        # Iterate over each time step
        for fieldname in n[f"/Step#0/Block/"].keys():
            data[fieldname] = n[f"/Step#0/Block/{fieldname}/0"][:,:-1,:-1] # there is extra point in the last dimension
            if filters is not None:
                if not isinstance(filters, list):
                    filters = [filters]
                for filteri in filters: # apply all filters in succession
                    if verbose:
                        print(f"Filtering {fieldname} from {filename} with {filteri['name']}")
                    filters_copy = filteri.copy()
                    filters_name = filters_copy.pop("name", None)
                    filters_object = getattr(nd, filters_name)
                    filter_kwargs = filters_copy
                    for _, kwarg in filter_kwargs.items():
                        if  isinstance(kwarg, list):
                            kwarg = tuple(kwarg)  #  configs usually provide lists, but we need tuples
                    data[fieldname] = filters_object(data[fieldname], **filter_kwargs)
                    if verbose:
                        print(f"Resulting shape {data[fieldname].shape}")
            if verbose:
                print(data[fieldname].shape)
            data[fieldname] = np.pad(data[fieldname], pad_width=((0,0), (0, 1), (0, 1)), mode='wrap')[0:1,...]
        
    with open(write_filename, 'wb') as out_file:
        pickle.dump(data, out_file)
    shutil.copy(f'{path}{read_folder}/SimuldationData.txt', f'{path}{write_folder}/SimuldationData.txt')

Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_000001.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_000001.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_006000.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_006000.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_007500.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_007500.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_009500.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_009500.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_002500.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_002500.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_004000.h5
Writing to /volume1/scratch/share_dir/peppe/data_filter/MSH-Fields_004000.h5.pkl
Processing /volume1/scratch/share_dir/peppe/data/MSH-Fields_011000.h5
Writing to /volume1/scra

In [5]:
path = '/volume1/scratch/share_dir/peppe/'
write_folder = 'data_filter'
read_folder = 'data'
import shutil
simulation_data_path = f'{path}{write_folder}/SimulationData.txt'
shutil.copy(f'{path}{read_folder}/SimulationData.txt', simulation_data_path)


# Read the file
with open(simulation_data_path, 'r') as file:
    lines = file.readlines()

# Modify the specific lines
for i, line in enumerate(lines):
    if 'Number of cells (x)' in line:
        lines[i] = 'Number of cells (x)      = 512\n'
    if 'Number of cells (y)' in line:
        lines[i] = 'Number of cells (y)      = 512\n'

# Write the modified lines back to the file
with open(simulation_data_path, 'w') as file:
    file.writelines(lines)

['MSH-Fields_006000.h5', 'MSH-Fields_007500.h5', 'MSH-Fields_009500.h5', 'MSH-Fields_002500.h5', 'MSH-Fields_004000.h5', 'MSH-Fields_011000.h5', 'MSH-Fields_000001.h5', 'MSH-Fields_012000.h5', 'MSH-Fields_008500.h5', 'MSH-Fields_013000.h5', 'MSH-Fields_003500.h5', 'MSH-Fields_004500.h5', 'MSH-Fields_005500.h5', 'MSH-Fields_014000.h5', 'MSH-Fields_000500.h5', 'MSH-Fields_002000.h5', 'MSH-Fields_001000.h5', 'MSH-Fields_006500.h5', 'MSH-Fields_010000.h5', 'MSH-Fields_014500.h5', 'MSH-Fields_015000.h5', 'MSH-Fields_007000.h5', 'MSH-Fields_010500.h5', 'MSH-Fields_003000.h5', 'MSH-Fields_013500.h5', 'MSH-Fields_008000.h5', 'MSH-Fields_012500.h5', 'MSH-Fields_005000.h5', 'MSH-Fields_001500.h5', 'MSH-Fields_011500.h5', 'MSH-Fields_009000.h5']
